## **Manejo de PySpqrk y SQL para entender el comportamiento de la tabla inicial**

Se utilizó PySpark y consultas SQL para explorar y analizar los datos relacionados con la atención en la sucursal. A través de estas herramientas, se identificaron patrones clave como la distribución horaria de la demanda, los tiempos de espera promedio, y los estados de los casos (completado o abandonado). Esto permitió construir una vista consolidada del sistema de atención, facilitando la posterior modelación y toma de decisiones para la impuración del nuevo archivo .csv generado en "001_impuracion-pandas.ipynb" y guardado en los archivos de entrada del proyectos como "datos_sucursal_imputado.csv".

### **Exploración de datos en crudo con SQL**

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, avg, hour, dayofweek, to_timestamp
from pyspark.sql.types import IntegerType, FloatType, StringType, TimestampType

In [3]:
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression

In [4]:
spark = SparkSession.builder \
    .appName("SucursalTurnosSQL") \
    .getOrCreate()

In [5]:
# Leer 
df = spark.read.option("header", True).option("inferSchema", True).csv('../files/input/Datos_Sucursal.csv')

In [6]:
df = df.withColumnRenamed("mes", "month").withColumnRenamed("dia", "day")

In [7]:
from pyspark.sql.functions import to_timestamp, hour

df = df.withColumn("hora_clean", to_timestamp("hora", "HH:mm:ss.SSS"))
df = df.withColumn("hora_num", hour("hora_clean"))

In [8]:
df.createOrReplaceTempView("sucursal_turnos")

In [9]:
# Ver el esquema del DataFrame para conocer los nombres de las columnas
df.printSchema()
print("Nombres de columnas:")
print(df.columns)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hora: timestamp (nullable = true)
 |-- usuario_encriptado: string (nullable = true)
 |-- tipo_servicio: string (nullable = true)
 |-- servicio: string (nullable = true)
 |-- sub_servicio: string (nullable = true)
 |-- tiempo_espera: string (nullable = true)
 |-- tiempo_servicio: string (nullable = true)
 |-- segm: string (nullable = true)
 |-- estado_caso: string (nullable = true)
 |-- asesor_encriptado: string (nullable = true)
 |-- hora_clean: timestamp (nullable = true)
 |-- hora_num: integer (nullable = true)

Nombres de columnas:
['year', 'month', 'day', 'hora', 'usuario_encriptado', 'tipo_servicio', 'servicio', 'sub_servicio', 'tiempo_espera', 'tiempo_servicio', 'segm', 'estado_caso', 'asesor_encriptado', 'hora_clean', 'hora_num']


No es correcto que: 

|-- tiempo_espera: string (nullable = true)

|-- tiempo_servicio: string (nullable = true)

Por lo que se debe realizar el cambio a tipo double por la forma (minutos), del campo.

In [10]:
df = df.withColumn("tiempo_espera", when(col("tiempo_espera").isin("NULL", ""), None).otherwise(col("tiempo_espera")))
df = df.withColumn("tiempo_servicio", when(col("tiempo_servicio").isin("NULL", ""), None).otherwise(col("tiempo_servicio")))

# cast
df = df.withColumn("tiempo_espera", col("tiempo_espera").cast("double"))
df = df.withColumn("tiempo_servicio", col("tiempo_servicio").cast("double"))

In [11]:
## Reemplazar la vista por los cambios realizados
df.createOrReplaceTempView("sucursal_turnos")

In [12]:
df.printSchema()
print("Nombres de columnas:")
print(df.columns)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hora: timestamp (nullable = true)
 |-- usuario_encriptado: string (nullable = true)
 |-- tipo_servicio: string (nullable = true)
 |-- servicio: string (nullable = true)
 |-- sub_servicio: string (nullable = true)
 |-- tiempo_espera: double (nullable = true)
 |-- tiempo_servicio: double (nullable = true)
 |-- segm: string (nullable = true)
 |-- estado_caso: string (nullable = true)
 |-- asesor_encriptado: string (nullable = true)
 |-- hora_clean: timestamp (nullable = true)
 |-- hora_num: integer (nullable = true)

Nombres de columnas:
['year', 'month', 'day', 'hora', 'usuario_encriptado', 'tipo_servicio', 'servicio', 'sub_servicio', 'tiempo_espera', 'tiempo_servicio', 'segm', 'estado_caso', 'asesor_encriptado', 'hora_clean', 'hora_num']


In [10]:
spark.sql("""DESCRIBE sucursal_turnos""").show()

+------------------+---------+-------+
|          col_name|data_type|comment|
+------------------+---------+-------+
|              year|      int|   NULL|
|             month|      int|   NULL|
|               day|      int|   NULL|
|              hora|timestamp|   NULL|
|usuario_encriptado|   string|   NULL|
|     tipo_servicio|   string|   NULL|
|          servicio|   string|   NULL|
|      sub_servicio|   string|   NULL|
|     tiempo_espera|   double|   NULL|
|   tiempo_servicio|   double|   NULL|
|              segm|   string|   NULL|
|       estado_caso|   string|   NULL|
| asesor_encriptado|   string|   NULL|
|        hora_clean|timestamp|   NULL|
|          hora_num|      int|   NULL|
+------------------+---------+-------+



Identificando los diferentes tipos de servicio, segmentos y demás campos, con el fin de imputar de manera correcta la información en el notebook-prueba:

In [11]:
spark.sql("""
SELECT DISTINCT servicio
FROM sucursal_turnos;
""").show() ## Sin Información

+--------------------+
|            servicio|
+--------------------+
|        Caja Divisas|
|         Caja Basica|
|       Asesoria Pyme|
|        Caja General|
|   Caja Preferencial|
|Asesoria Preferen...|
|         Autogestión|
|    Asesoria General|
|     Asesoria Turnos|
|        Caja Turnos |
|                NULL|
|    Caja Prioritaria|
|         Caja Masiva|
+--------------------+



In [15]:
spark.sql("""
SELECT DISTINCT segm
FROM sucursal_turnos;
""").show() ## Sin Segmento

+--------------------+
|                segm|
+--------------------+
|         Corporativo|
|            Personal|
|     Segmento Social|
|         Empresarial|
|        Preferencial|
|           Micropyme|
|       Personal Plus|
|    Constructor Pyme|
|                Pyme|
|                NULL|
|Constructor Corpo...|
|     Gobierno De Red|
+--------------------+



In [16]:
spark.sql("""
SELECT
    year, 
    month, 
    day, 
    hora, 
    usuario_encriptado, 
    tipo_servicio, 
    servicio, 
    sub_servicio,
    tiempo_espera, 
    tiempo_servicio, 
    segm, 
    estado_caso, 
    asesor_encriptado
FROM sucursal_turnos
LIMIT 10;
""").show()

+----+-----+---+--------------------+--------------------+-------------+-----------------+--------------------+------------------+------------------+-------------+-----------+--------------------+
|year|month|day|                hora|  usuario_encriptado|tipo_servicio|         servicio|        sub_servicio|     tiempo_espera|   tiempo_servicio|         segm|estado_caso|   asesor_encriptado|
+----+-----+---+--------------------+--------------------+-------------+-----------------+--------------------+------------------+------------------+-------------+-----------+--------------------+
|2024|    1|  2|2025-07-26 15:34:...|20aed4d067c167cbd...|     Asesoria| Asesoria General|    Asesoria General| 34.46666666666667|             12.35|     Personal| Completado|d59d2bb716c503bcb...|
|2024|    1|  2|2025-07-26 10:41:...|2633c62c47bcb340c...|         Caja|Caja Preferencial|Caja Preferencial...| 95.08333333333333|              9.95|Personal Plus| Completado|09c9a38a05f3d4da9...|
|2024|    1|  2

In [ ]:
spark.sql("""
SELECT
    year, 
    month, 
    day, 
    hora, 
    usuario_encriptado, 
    tipo_servicio, 
    servicio, 
    sub_servicio,
    tiempo_espera, 
    tiempo_servicio, 
    segm, 
    estado_caso, 
    asesor_encriptado
FROM sucursal_turnos
WHERE tiempo_espera = 0
LIMIT 20;
""").show()

+----+-----+---+--------------------+--------------------+-------------+--------------------+--------------------+-------------+------------------+-------------+-----------+--------------------+
|year|month|day|                hora|  usuario_encriptado|tipo_servicio|            servicio|        sub_servicio|tiempo_espera|   tiempo_servicio|         segm|estado_caso|   asesor_encriptado|
+----+-----+---+--------------------+--------------------+-------------+--------------------+--------------------+-------------+------------------+-------------+-----------+--------------------+
|2024|    1|  4|2025-07-27 11:01:...|bf9d42906c76d0074...|     Asesoria|    Asesoria General|    Asesoria General|          0.0| 7.366666666666666|    Micropyme| Completado|3c2de0b7122c939c3...|
|2024|    1| 12|2025-07-27 11:56:...|8ded3ce5bf5145213...|     Asesoria|     Asesoria Turnos|     Asesoria Turnos|          0.0|              46.3|    Micropyme| Completado|787435b566c81a349...|
|2024|    1| 19|2025-07-2

In [ ]:
spark.sql("""
SELECT
    COUNT(*) AS total_turnos
FROM sucursal_turnos
WHERE tiempo_espera = 0
LIMIT 20;
""").show() ## Podrían ser errores de captura de información, relativamente pocos casos

+------------+
|total_turnos|
+------------+
|          62|
+------------+



In [ ]:
spark.sql("""
SELECT
    COUNT(*) AS total_turnos
FROM sucursal_turnos
WHERE tiempo_servicio = 0
LIMIT 20;
""").show() ## Podrían ser errores de captura de información, relativamente pocos casos

+------------+
|total_turnos|
+------------+
|          35|
+------------+



In [25]:
spark.sql("""
SELECT
    year, 
    month, 
    day, 
    hora, 
    usuario_encriptado, 
    tipo_servicio, 
    servicio, 
    sub_servicio,
    tiempo_espera, 
    tiempo_servicio, 
    segm, 
    estado_caso, 
    asesor_encriptado
FROM sucursal_turnos
WHERE tiempo_servicio IS NULL AND estado_caso = 'Completado'
LIMIT 20;
""").show()

+----+-----+---+--------------------+--------------------+--------------------+-----------------+--------------------+-------------------+---------------+---------------+-----------+--------------------+
|year|month|day|                hora|  usuario_encriptado|       tipo_servicio|         servicio|        sub_servicio|      tiempo_espera|tiempo_servicio|           segm|estado_caso|   asesor_encriptado|
+----+-----+---+--------------------+--------------------+--------------------+-----------------+--------------------+-------------------+---------------+---------------+-----------+--------------------+
|2024|    1|  2|2025-07-27 14:46:...|35eacd33ea7ffd3fd...|Canales Alternativos|      Autogestión| Sucursal Telefónica|  43.46666666666667|           NULL|       Personal| Completado|8267f1bcf1d67f35e...|
|2024|    1|  2|2025-07-27 15:51:...|80dedeec03dbdf1e8...|                Caja|     Caja General|        Caja General|              66.35|           NULL|       Personal| Completado|28

In [26]:
spark.sql("""
SELECT
    COUNT(*)
FROM sucursal_turnos
WHERE tiempo_servicio IS NULL AND estado_caso = 'Completado'
LIMIT 20;
""").show()

+--------+
|count(1)|
+--------+
|    6386|
+--------+



In [27]:
spark.sql("""
SELECT
    COUNT(*)
FROM sucursal_turnos
WHERE tiempo_servicio IS NULL AND estado_caso = 'Abandonado'
LIMIT 20;
""").show()

+--------+
|count(1)|
+--------+
|   13562|
+--------+



### **Resumen de tiempos**

In [19]:
spark.sql("""
SELECT 
  AVG(tiempo_espera) AS avg_espera,
  AVG(tiempo_servicio) AS avg_servicio
FROM sucursal_turnos
""").show()

+-----------------+------------------+
|       avg_espera|      avg_servicio|
+-----------------+------------------+
|33.34095483087441|13.529750245673627|
+-----------------+------------------+



*Nota: no ejecutable de aquí para abajo por el tipado de las variables*

### **Tiempos por tipo de servicio**

In [ ]:
spark.sql("""
SELECT 
  tipo_servicio,
  ROUND(AVG(tiempo_espera), 2) AS avg_espera,
  ROUND(AVG(tiempo_servicio), 2) AS avg_servicio
FROM turnos
GROUP BY tipo_servicio
ORDER BY avg_espera DESC
""").show()

### **Tiempos por segmento**

In [ ]:
spark.sql("""
SELECT 
  segm,
  ROUND(AVG(tiempo_espera), 2) AS avg_espera,
  ROUND(AVG(tiempo_servicio), 2) AS avg_servicio
FROM turnos
GROUP BY segm
ORDER BY avg_espera DESC
""").show()

### **Volumen por hora**

In [ ]:
spark.sql("""
SELECT 
  hora_num,
  COUNT(*) AS atenciones
FROM turnos
GROUP BY hora_num
ORDER BY hora_num
""").show()